In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_1_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_csv/"+'gcc6'+"_1_32.csv", index_col=0)

# 형태 출력
print(gcc6_1_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_1_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_1_32.head()

(1811837, 2)
reset_index 완료
input data shape


,hex,bin
0,55,1
1,57,0
2,56,0
3,53,0
4,83,0


In [2]:
gcc6_1_32.tail()

,hex,bin
1811832,c3,0
1811833,8b,1
1811834,1c,0
1811835,24,0
1811836,c3,0


In [3]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_1_32['hex'].unique()))

# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_1_32_onehot = pd.get_dummies(gcc6_1_32)


print('원핫인코딩완료')

print(gcc6_1_32_onehot.shape)

# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_1_32_onehot['bin'].value_counts())

256
원핫인코딩완료
(1811837, 257)
0    1800653
1      11184
Name: bin, dtype: int64


In [4]:
# (5-1) gcc3 6gram
######################## 
idx3 = gcc6_1_32_onehot[gcc6_1_32_onehot['bin']==1].index  # 407, 474 ...
ls3 = list(idx3)

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 
left_idx3, right_idx3 = 0, 7 # 3개씩

# 6gram
for k in range(left_idx3, right_idx3):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능
print(len(ls_idx3))

#ls_idx3 = list(set(ls_idx3)) 
print(len(ls_idx3))

ls_idx3.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx3 = list(filter(lambda x: x<len(gcc6_1_32_onehot), ls_idx3))
print(len(ls_idx3))

# 2차 남은 index들 중 right_idx3 나눈 나머지 없애기
sub = len(ls_idx3)%(right_idx3)
print(sub)

ls_idx3 = ls_idx3[:-4]
print(len(ls_idx3))

print('gcc6_1_32', len(ls_idx3))

# loc 로 수정필요
gcc6_1_32_onehot_3gram = gcc6_1_32_onehot.loc[ls_idx3,:].copy()

78288
78288
78285
4
78281
gcc6_1_32 78281


In [5]:
# 훈련 데이터, 훈련 라벨
x_gcc6_1_32_3 = gcc6_1_32_onehot_3gram.iloc[:,1:].to_numpy()
y_gcc6_1_32_3 = gcc6_1_32_onehot_3gram['bin'].to_numpy()
print(x_gcc6_1_32_3.shape, y_gcc6_1_32_3.shape)


(78281, 256) (78281,)


In [6]:
x_gcc6_1_32_3 = x_gcc6_1_32_3.reshape(-1, right_idx3, x_gcc6_1_32_3.shape[1])
y_gcc6_1_32_3 = y_gcc6_1_32_3.reshape(-1, right_idx3, 1)

print(x_gcc6_1_32_3.shape, y_gcc6_1_32_3.shape)

(11183, 7, 256) (11183, 7, 1)


## 모델

In [7]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None,right_idx3, 256)) 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model1 = Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7, 256)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 7, 32)             34944     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 1)              33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________


In [8]:
########## 3gram

# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc6_1_32_3, y_gcc6_1_32_3):
    print('======Training stage======')
    model1.fit(x_gcc6_1_32_3[train],
               y_gcc6_1_32_3[train],
               epochs = 30,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc6_1_32_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_1_32_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

======Training stage======
Epoch 1/30
10064/10064 [==============================] - 4s 419us/step - loss: 0.2222 - accuracy: 0.9206
Epoch 2/30
10064/10064 [==============================] - 3s 253us/step - loss: 0.0405 - accuracy: 0.9860
Epoch 3/30
10064/10064 [==============================] - 3s 259us/step - loss: 0.0270 - accuracy: 0.9921
Epoch 4/30
10064/10064 [==============================] - 3s 257us/step - loss: 0.0196 - accuracy: 0.9950
Epoch 5/30
10064/10064 [==============================] - 2s 247us/step - loss: 0.0156 - accuracy: 0.9959
Epoch 6/30
10064/10064 [==============================] - 3s 259us/step - loss: 0.0137 - accuracy: 0.9964
Epoch 7/30
10064/10064 [==============================] - 3s 254us/step - loss: 0.0127 - accuracy: 0.9965
Epoch 8/30
10064/10064 [==============================] - 3s 279us/step - loss: 0.0117 - accuracy: 0.9969
Epoch 9/30
10064/10064 [==============================] - 3s 295us/step - loss: 0.0109 - accuracy: 0.9969
Epoch 10/30
10064/1

accuracy_score 0.9943827396910507
recall_score 0.9852216748768473
precision_score 0.9787928221859706
f1_score 0.9819967266775778
======Training stage======
Epoch 1/30
10064/10064 [==============================] - 3s 332us/step - loss: 0.0078 - accuracy: 0.9982
Epoch 2/30
10064/10064 [==============================] - 4s 352us/step - loss: 0.0075 - accuracy: 0.99810s - loss: 0
Epoch 3/30
10064/10064 [==============================] - 4s 353us/step - loss: 0.0074 - accuracy: 0.9980
Epoch 4/30
10064/10064 [==============================] - 3s 345us/step - loss: 0.0071 - accuracy: 0.9982
Epoch 5/30
10064/10064 [==============================] - 3s 339us/step - loss: 0.0068 - accuracy: 0.9982
Epoch 6/30
10064/10064 [==============================] - 3s 342us/step - loss: 0.0068 - accuracy: 0.9982
Epoch 7/30
10064/10064 [==============================] - 4s 354us/step - loss: 0.0066 - accuracy: 0.9982
Epoch 8/30
10064/10064 [==============================] - 4s 351us/step - loss: 0.0066 - a

accuracy_score 0.9989786799438274
recall_score 0.9949195596951735
precision_score 0.9983007646559049
f1_score 0.996607294317218
======Training stage======
Epoch 1/30
10064/10064 [==============================] - 3s 327us/step - loss: 0.0036 - accuracy: 0.9990
Epoch 2/30
10064/10064 [==============================] - 3s 324us/step - loss: 0.0034 - accuracy: 0.9991
Epoch 3/30
10064/10064 [==============================] - 3s 327us/step - loss: 0.0034 - accuracy: 0.9990
Epoch 4/30
10064/10064 [==============================] - 3s 324us/step - loss: 0.0032 - accuracy: 0.9991
Epoch 5/30
10064/10064 [==============================] - 3s 340us/step - loss: 0.0031 - accuracy: 0.9991
Epoch 6/30
10064/10064 [==============================] - 3s 343us/step - loss: 0.0030 - accuracy: 0.9992
Epoch 7/30
10064/10064 [==============================] - ETA: 0s - loss: 0.0030 - accuracy: 0.99 - 3s 331us/step - loss: 0.0030 - accuracy: 0.9991
Epoch 8/30
10064/10064 [==============================] - 3s 

accuracy_score 0.9931060896208349
recall_score 0.9799839871897518
precision_score 0.9768555466879489
f1_score 0.9784172661870504
======Training stage======
Epoch 1/30
10065/10065 [==============================] - 2s 233us/step - loss: 0.0043 - accuracy: 0.9988
Epoch 2/30
10065/10065 [==============================] - 2s 230us/step - loss: 0.0036 - accuracy: 0.9989
Epoch 3/30
10065/10065 [==============================] - 2s 231us/step - loss: 0.0033 - accuracy: 0.9990
Epoch 4/30
10065/10065 [==============================] - 2s 232us/step - loss: 0.0031 - accuracy: 0.9990
Epoch 5/30
10065/10065 [==============================] - 2s 233us/step - loss: 0.0029 - accuracy: 0.9991
Epoch 6/30
10065/10065 [==============================] - 2s 233us/step - loss: 0.0029 - accuracy: 0.9991
Epoch 7/30
10065/10065 [==============================] - 2s 233us/step - loss: 0.0027 - accuracy: 0.9991
Epoch 8/30
10065/10065 [==============================] - 2s 228us/step - loss: 0.0027 - accuracy: 0.9

accuracy_score 0.9978277536417072
recall_score 0.9869791666666666
precision_score 0.9982440737489026
f1_score 0.9925796595373199
======Training stage======
Epoch 1/30
10065/10065 [==============================] - 2s 228us/step - loss: 0.0016 - accuracy: 0.9995
Epoch 2/30
10065/10065 [==============================] - 2s 228us/step - loss: 0.0015 - accuracy: 0.9996
Epoch 3/30
10065/10065 [==============================] - 2s 229us/step - loss: 0.0014 - accuracy: 0.9996
Epoch 4/30
10065/10065 [==============================] - 2s 228us/step - loss: 0.0013 - accuracy: 0.9996
Epoch 5/30
10065/10065 [==============================] - 2s 228us/step - loss: 0.0013 - accuracy: 0.9996
Epoch 6/30
10065/10065 [==============================] - 2s 228us/step - loss: 0.0011 - accuracy: 0.9997
Epoch 7/30
10065/10065 [==============================] - 2s 229us/step - loss: 0.0010 - accuracy: 0.9997
Epoch 8/30
10065/10065 [==============================] - 2s 227us/step - loss: 0.0011 - accuracy: 0.9

...

accuracy_score 0.9998722208024534
recall_score 1.0
precision_score 0.9991197183098591
f1_score 0.9995596653456628
======Training stage======
Epoch 1/30
10065/10065 [==============================] - 2s 233us/step - loss: 6.2470e-04 - accuracy: 0.9998
Epoch 2/30
10065/10065 [==============================] - 2s 231us/step - loss: 5.6165e-04 - accuracy: 0.9998
Epoch 3/30
10065/10065 [==============================] - 2s 231us/step - loss: 5.2720e-04 - accuracy: 0.9998
Epoch 4/30
10065/10065 [==============================] - 2s 230us/step - loss: 5.0039e-04 - accuracy: 0.9999
Epoch 5/30
10065/10065 [==============================] - 2s 228us/step - loss: 5.0074e-04 - accuracy: 0.9999
Epoch 6/30
10065/10065 [==============================] - 2s 231us/step - loss: 5.0761e-04 - accuracy: 0.9999
Epoch 7/30
10065/10065 [==============================] - 2s 230us/step - loss: 4.7153e-04 - accuracy: 0.9999
Epoch 8/30
10065/10065 [==============================] - 2s 228us/step - loss: 4.6161e-0

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
10065/10065 [==============================] - 2s 234us/step - loss: 3.7603e-04 - accuracy: 0.9999
Epoch 2/30
10065/10065 [==============================] - 2s 235us/step - loss: 3.5669e-04 - accuracy: 0.9999
Epoch 3/30
10065/10065 [==============================] - 2s 237us/step - loss: 3.6294e-04 - accuracy: 0.9999
Epoch 4/30
10065/10065 [==============================] - 2s 245us/step - loss: 3.8609e-04 - accuracy: 0.9999
Epoch 5/30
10065/10065 [==============================] - 2s 245us/step - loss: 3.7786e-04 - accuracy: 0.9999
Epoch 6/30
10065/10065 [==============================] - 2s 246us/step - loss: 3.7696e-04 - accuracy: 0.9999
Epoch 7/30
10065/10065 [==============================] - 2s 240us/step - loss: 3.9605e-04 - accuracy: 0.9999
Epoch 8/30
10065/10065 [==============================] - 2s 237us/step - loss: 4.1793e-04 - accuracy: 0.9999
Epoch 9/30
10065/10065 [

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
10065/10065 [==============================] - 2s 232us/step - loss: 3.1237e-04 - accuracy: 0.9999
Epoch 2/30
10065/10065 [==============================] - 2s 237us/step - loss: 3.4312e-04 - accuracy: 0.9999
Epoch 3/30
10065/10065 [==============================] - 2s 235us/step - loss: 3.5953e-04 - accuracy: 0.9999
Epoch 4/30
10065/10065 [==============================] - 2s 230us/step - loss: 3.9509e-04 - accuracy: 0.9999
Epoch 5/30
10065/10065 [==============================] - 2s 230us/step - loss: 3.6446e-04 - accuracy: 0.9999
Epoch 6/30
10065/10065 [==============================] - 2s 230us/step - loss: 3.5939e-04 - accuracy: 0.9999
Epoch 7/30
10065/10065 [==============================] - 2s 230us/step - loss: 3.6363e-04 - accuracy: 0.9999
Epoch 8/30
10065/10065 [==============================] - 2s 239us/step - loss: 3.5341e-04 - accuracy: 0.9999
Epoch 9/30
10065/10065 [

accuracy_score 0.9996166624073601
recall_score 0.9974726200505476
precision_score 1.0
f1_score 0.9987347110923661
======Training stage======
Epoch 1/30
10065/10065 [==============================] - 2s 196us/step - loss: 3.7504e-04 - accuracy: 0.9999
Epoch 2/30
10065/10065 [==============================] - 2s 198us/step - loss: 3.0055e-04 - accuracy: 0.9999
Epoch 3/30
10065/10065 [==============================] - 2s 197us/step - loss: 2.7650e-04 - accuracy: 0.9999
Epoch 4/30
10065/10065 [==============================] - 2s 196us/step - loss: 2.5487e-04 - accuracy: 1.0000
Epoch 5/30
10065/10065 [==============================] - 2s 190us/step - loss: 2.3125e-04 - accuracy: 1.0000
Epoch 6/30
10065/10065 [==============================] - 2s 172us/step - loss: 2.3989e-04 - accuracy: 1.0000
Epoch 7/30
10065/10065 [==============================] - 2s 172us/step - loss: 2.5402e-04 - accuracy: 1.0000
Epoch 8/30
10065/10065 [==============================] - 2s 173us/step - loss: 2.1809e-0

accuracy_score 0.9997444416049067
recall_score 0.9983579638752053
precision_score 1.0
f1_score 0.9991783073130649

K-fold cross validation Accuracy: [0.9943827396910507, 0.9989786799438274, 0.9931060896208349, 0.9978277536417072, 0.9973166368515206, 0.9998722208024534, 1.0, 1.0, 0.9996166624073601, 0.9997444416049067]

K-fold cross validation Recall: [0.9852216748768473, 0.9949195596951735, 0.9799839871897518, 0.9869791666666666, 0.9943639291465378, 1.0, 1.0, 1.0, 0.9974726200505476, 0.9983579638752053]

K-fold cross validation Precision: [0.9787928221859706, 0.9983007646559049, 0.9768555466879489, 0.9982440737489026, 0.988791032826261, 0.9991197183098591, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9819967266775778, 0.996607294317218, 0.9784172661870504, 0.9925796595373199, 0.9915696507426736, 0.9995596653456628, 1.0, 1.0, 0.9987347110923661, 0.9991783073130649]


In [9]:
# 4gram 평가지표
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9980845224563663
10-Fold Cross_validation. Recall : 0.9937298901500731
10-Fold Cross_validation. Precision : 0.9940103958414847
10-Fold Cross_validation. F1-Score : 0.9938643281212933
